In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

train_df=pd.read_csv("/content/drive/MyDrive/kaggle-study/1/kisti-sejong-winterschool-p3/train.csv")
test_df =pd.read_csv("/content/drive/MyDrive/kaggle-study/1/kisti-sejong-winterschool-p3/test.csv")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             244 non-null    int64  
 1   OC                  244 non-null    object 
 2   sido                244 non-null    object 
 3   sgg                 244 non-null    int64  
 4   openDate            244 non-null    int64  
 5   bedCount            239 non-null    float64
 6   instkind            243 non-null    object 
 7   revenue1            240 non-null    float64
 8   salescost1          240 non-null    float64
 9   sga1                240 non-null    float64
 10  salary1             240 non-null    float64
 11  noi1                240 non-null    float64
 12  noe1                240 non-null    float64
 13  interest1           240 non-null    float64
 14  ctax1               240 non-null    float64
 15  profit1             240 non-null    float64
 16  liquidAs

In [4]:
train_df.OC= train_df.OC.astype('str').str.replace(" ","")

In [5]:
train_test_prod = train_df.append(test_df)

In [6]:
train_test_prod.shape

(301, 58)

In [7]:
def split_openDate_to_ymd(X):
    X['openDate'] = X['openDate'].astype(str)
    X['openYear'] = X['openDate'].str[:4]
    X['openMonth'] = X['openDate'].str[4:6]
    X['openDay'] = X['openDate'].str[6:]
    
    X = X.drop('openDate', axis=1)
    return X

In [8]:
train_test_prod = split_openDate_to_ymd(train_test_prod)
train_test_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 0 to 56
Data columns (total 60 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  244 non-null    object 
 2   sido                301 non-null    object 
 3   sgg                 301 non-null    int64  
 4   bedCount            296 non-null    float64
 5   instkind            300 non-null    object 
 6   revenue1            293 non-null    float64
 7   salescost1          293 non-null    float64
 8   sga1                293 non-null    float64
 9   salary1             293 non-null    float64
 10  noi1                293 non-null    float64
 11  noe1                293 non-null    float64
 12  interest1           293 non-null    float64
 13  ctax1               293 non-null    float64
 14  profit1             293 non-null    float64
 15  liquidAsset1        293 non-null    float64
 16  quickAsse

In [9]:
# sgg 도 범주형 데이터 이므로
train_test_prod['sgg'] = train_test_prod['sgg'].astype(str)

In [10]:
factor_columns = train_test_prod.select_dtypes(include = ['object']).columns
numeric_columns = train_test_prod.columns.difference(factor_columns)
train_test_prod["OC"] = train_test_prod["OC"].fillna("Don't_know")

train_test_prod[factor_columns] = train_test_prod[factor_columns].fillna('Not_sure')

train_test_prod[numeric_columns] = train_test_prod[numeric_columns].fillna(-999)

In [11]:
factor_columns

Index(['OC', 'sido', 'sgg', 'instkind', 'ownerChange', 'openYear', 'openMonth',
       'openDay'],
      dtype='object')

In [12]:
label_columns = ['OC', 'ownerChange']
onehot_columns = [col for col in factor_columns if col not in label_columns]

In [13]:
from sklearn.preprocessing import LabelEncoder
fac_le = LabelEncoder()
train_test_prod[label_columns] = train_test_prod.loc[:,label_columns].apply(lambda x : fac_le.fit_transform(x))

In [14]:
onehot_encoded = pd.get_dummies(train_test_prod[onehot_columns])
train_test_prod = pd.concat([train_test_prod, onehot_encoded], axis=1)
train_test_prod = train_test_prod.drop(onehot_columns, axis=1)

In [15]:
dep = 'OC'
indep = train_test_prod.columns.difference([dep])
train_prod = train_test_prod.loc[train_test_prod.OC != 0,]
test_prod = train_test_prod.loc[train_test_prod.OC == 0,]
train_prod.OC=train_prod.OC-1

sub_id = test_prod.inst_id

train_prod_X = train_prod[indep]
train_prod_Y = train_prod[dep]
test_prod_X = test_prod[indep]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
scaler=StandardScaler()
svm=SVC(gamma="auto",probability=True)
train_prod_X_sc=scaler.fit_transform(train_prod_X)
test_prod_X_sc=scaler.transform(test_prod_X)

param_grid = [
  {'C': [0.001,0.01,0.1,1, 10, 100, 1000], 'kernel': ['rbf','linear']},
 ]
search = GridSearchCV(svm, param_grid,cv=5)
search.fit(train_prod_X_sc, train_prod_Y)
predict=search.predict(test_prod_X_sc)

In [17]:
sub_RF = pd.DataFrame({'inst_id' : sub_id , 'OC' : predict })
sub_RF = sub_RF[['inst_id', 'OC']]
sub_RF.to_csv("result_SVM.csv",index=False)

In [19]:
onehot_encoded

,sido_busan,sido_choongbuk,sido_choongnam,sido_daegu,sido_daejeon,sido_gangwon,sido_gwangju,sido_gyeongbuk,sido_gyeonggi,sido_gyeongnam,sido_incheon,sido_jeonbuk,sido_jeonnam,sido_sejong,sido_seoul,sido_ulsan,sgg_1,sgg_100,sgg_101,sgg_102,sgg_103,sgg_104,sgg_106,sgg_107,sgg_108,sgg_109,sgg_11,sgg_110,sgg_111,sgg_112,sgg_113,sgg_114,sgg_115,sgg_117,sgg_118,sgg_119,sgg_12,sgg_120,sgg_121,sgg_123,...,openMonth_04,openMonth_05,openMonth_06,openMonth_07,openMonth_08,openMonth_09,openMonth_10,openMonth_11,openMonth_12,openDay_01,openDay_02,openDay_03,openDay_04,openDay_05,openDay_06,openDay_07,openDay_08,openDay_09,openDay_10,openDay_11,openDay_12,openDay_13,openDay_14,openDay_15,openDay_16,openDay_17,openDay_18,openDay_19,openDay_20,openDay_21,openDay_22,openDay_23,openDay_24,openDay_25,openDay_26,openDay_27,openDay_28,openDay_29,openDay_30,openDay_31
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
onehot_columns

['sido', 'sgg', 'instkind', 'openYear', 'openMonth', 'openDay']